<a href="https://colab.research.google.com/github/Bryan-Az/TransformerLLM-Finetuning/blob/main/NanoGPT_TransformerLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A 'Nano' Generative Pre-trained Transformer LLM
This is an experiment where I create and train a mini GPT model from scratch. It will be using a ebook of my choice as its' training data.

## Imports and Installs

In [1]:
import gdown

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Extracting, Transforming, and Loading the Ebook Data
The book I chose is "Voices from within the Veil" by W.E.B DuBois. This book was taken from the Project Gutenberg website and is available to use freely. I uploaded it to Google Drive for my own use as to not scrape project gutenberg's website.

In [4]:
voices_within_veil_glink = 'https://drive.google.com/file/d/1Ah7n0r-bjW8nbTdc82C8_9_4l3c9lOql/view?usp=sharing'
voices_within_veil_id = voices_within_veil_glink.split('/')[-2]
voices_within_veil_path = gdown.download(id=voices_within_veil_id)

Downloading...
From: https://drive.google.com/uc?id=1Ah7n0r-bjW8nbTdc82C8_9_4l3c9lOql
To: /content/Voices_from_within_the_Veil_WEBDuBois.txt
100%|██████████| 419k/419k [00:00<00:00, 108MB/s]


In [5]:
voices_within_veil_lines = open(voices_within_veil_path, 'r').readlines()
voices_within_veil = ''.join(voices_within_veil_lines)

In [6]:
print(voices_within_veil[50:305])

from Within the Veil
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project 


In [7]:
chars = sorted(list(set(voices_within_veil)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzàäçéëñô—‘’“”•™﻿
98


In [33]:
# creating a tokenizer using a simple encoding function
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [34]:
# encoding the data
encoded_voices_within_veil = []
for line in voices_within_veil_lines:
    encoded_voices_within_veil.append(encode(line))

In [35]:
# concatenating all lists to become a single matrix
encoded_voices_within_veil = [item for sublist in encoded_voices_within_veil for item in sublist]
encoded_voices_within_veil = torch.tensor(encoded_voices_within_veil)

In [36]:
# saving the ebook into train and test splits
percent_train = 0.9
train_data = encoded_voices_within_veil[:int(percent_train*len(encoded_voices_within_veil))]
test_data = encoded_voices_within_veil[int(percent_train*len(encoded_voices_within_veil)):]

In [37]:
f"Size of train, test splits: {len(train_data), len(test_data)}"

'Size of train, test splits: (369613, 41069)'

In [38]:
# sent the train and test data to gpu device
train_data = train_data.to(device)
test_data = test_data.to(device)

In [39]:
# function to batch the data so that the first N words are 'context' and the final N+1 word is the 'target'.
batch_size = 4 # independent sequences processed in parallel
block_size = 8 # maximum context length for predictions

def get_batch(split):
    # generate a small batch; inputs x and targets y
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size - 3): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1, 57, 62, 71, 71, 76,  1, 76],
        [71, 74, 13,  1, 46, 77, 59, 64],
        [64, 61,  1, 60, 65, 75, 57, 60],
        [76, 75, 13,  1, 41, 71, 79, 11]], device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[57, 62, 71, 71, 76,  1, 76, 64],
        [74, 13,  1, 46, 77, 59, 64,  1],
        [61,  1, 60, 65, 75, 57, 60, 78],
        [75, 13,  1, 41, 71, 79, 11,  1]], device='cuda:0')
----
when input is [1] the target: 57
when input is [1, 57] the target: 62
when input is [1, 57, 62] the target: 71
when input is [1, 57, 62, 71] the target: 71
when input is [1, 57, 62, 71, 71] the target: 76
when input is [1, 57, 62, 71, 71, 76] the target: 1
when input is [1, 57, 62, 71, 71, 76, 1] the target: 76
when input is [1, 57, 62, 71, 71, 76, 1, 76] the target: 64


## Initializing the Mini Transformer Model

In [40]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [41]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

In [42]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [43]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [44]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [45]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        idx = torch.clamp(idx, 0, self.token_embedding_table.num_embeddings - 1)
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # Ensure targets are within the valid range for logits
            targets = torch.clamp(targets, 0, C - 1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

## Training the Mini Transformer LLM with Self-Attention

In [46]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [47]:
nanoGPT_model = BigramLanguageModel()
nanoGPT_model = nanoGPT_model.to(device)

In [48]:
# print the number of parameters in the model
print(sum(p.numel() for p in nanoGPT_model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(nanoGPT_model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(nanoGPT_model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = nanoGPT_model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

0.213986 M parameters
step 0: train loss 4.8444, val loss 4.8433
step 100: train loss 2.6641, val loss 2.8629
step 200: train loss 2.5499, val loss 2.7662
step 300: train loss 2.4754, val loss 2.6891
step 400: train loss 2.3962, val loss 2.5912
step 500: train loss 2.3192, val loss 2.5376
step 600: train loss 2.2601, val loss 2.4871
step 700: train loss 2.2198, val loss 2.4073
step 800: train loss 2.1797, val loss 2.3997
step 900: train loss 2.1432, val loss 2.3641
step 1000: train loss 2.1093, val loss 2.3112
step 1100: train loss 2.0811, val loss 2.3027
step 1200: train loss 2.0566, val loss 2.2644
step 1300: train loss 2.0333, val loss 2.2401
step 1400: train loss 1.9975, val loss 2.2205
step 1500: train loss 1.9759, val loss 2.2088
step 1600: train loss 1.9546, val loss 2.2013
step 1700: train loss 1.9435, val loss 2.1963
step 1800: train loss 1.9313, val loss 2.1679
step 1900: train loss 1.9066, val loss 2.1710
step 2000: train loss 1.8982, val loss 2.1394
step 2100: train loss 1.

## Generating Text Completion from the Nano-GPT Model

In [28]:
input = 'Voice from'

In [49]:
context = torch.tensor(encode(input), dtype=torch.long, device=device).unsqueeze(0)

In [50]:
print(decode(nanoGPT_model.generate(context, max_new_tokens=20)[0].tolist()))

Voice from colonuage, with it 
